### Get stock prices data from polygon API

In [5]:
#!pip install -r requirements.txt

In [6]:
from stockify import * 
import datetime as dt 
from polygon import RESTClient
from polygon.rest import models
    
API_key = "PuJHla6_U_GDHcNE3XSpk7L2gCCpdJ9J"
client = RESTClient(API_key)

In [8]:
## Put the API key (if we do pay) into ENV_VARS
current_date = dt.datetime.now().date()
stocks_tickers = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=True)

for st in stocks_tickers:
    
    #try:
        raw_response = get_p_process_by_ticker(API_key,
                                            client,
                                            st,
                                            get_daily_time_buckets(max_lookback_years=2,
                                                                    current_is_timestamp=False,
                                                                    frequency='daily',
                                                                    include_weekend=False),
                                                stock_prices_frequency= 'daily')
        
        df = get_jsonList_into_dataframe(get_serialized_raw_into_json(raw_response,st),
                                    ['close'])
    #except (NoResultsError) as e:
    #    continue

In [11]:
df

,stock,converted_utc_timestamp,close
0,AAPL,2020-12-28 05:00:00,136.69
1,AAPL,2020-12-29 05:00:00,134.87
2,AAPL,2020-12-30 05:00:00,133.72
3,AAPL,2020-12-31 05:00:00,132.69
4,AAPL,2021-01-04 05:00:00,129.41
...,...,...,...
498,AAPL,2022-12-19 05:00:00,132.37
499,AAPL,2022-12-20 05:00:00,132.30
500,AAPL,2022-12-21 05:00:00,135.45
501,AAPL,2022-12-22 05:00:00,132.23


In [10]:
stocks_tickers

['AAPL']